## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

### Load Dataset Files

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [0]:
project_path = "/content/drive/My Drive/dog-breed-identification/"

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels in further steps.

In [0]:
#Commenting the below code as the Files are already in unzipped Format
#from zipfile import ZipFile
#with ZipFile(project_path+'train.zip', 'r') as z:
#  z.extractall()

Repeat the same step for test.zip

In [0]:
#It's already in unzipped Format

Repeat the same step for sample_submission.csv.zip

In [0]:
#It's already in unzipped Format

Repeat the same step for labels.csv.zip

In [0]:
#It's already in unzipped Format

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [0]:
import pandas as pd

In [0]:
labels_df = pd.read_csv(project_path + 'labels.csv')

### Print the count of each category of Dogs given in the dataset



In [9]:
labels_df['breed'].value_counts()

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
                       ... 
golden_retriever         67
brabancon_griffon        67
komondor                 67
briard                   66
eskimo_dog               66
Name: breed, Length: 120, dtype: int64

### Get one-hot encodings of labels

In [0]:
import numpy as np

In [0]:
labels = np.array(pd.get_dummies(labels_df['breed']))

## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [0]:
img_rows = 128
img_columns = 128
num_channels = 3

In [13]:
from tqdm import tqdm
import cv2
x_train = []
y_train = []

for i, breed in tqdm(labels_df.values):
  train_img = cv2.imread(project_path + 'train/{}.jpg'.format(i), 1)
  train_img_resize = cv2.resize(train_img, (img_rows, img_columns))
  x_train.append(train_img_resize)
  y_train.append(breed)

100%|██████████| 10222/10222 [01:22<00:00, 124.20it/s]


Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [0]:
import numpy as np

x_train = np.array(x_train)
x_train = x_train / 255

x_train = np.reshape(x_train, (len(x_train), img_rows, img_columns, num_channels))

### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [0]:
from sklearn.model_selection import train_test_split

(X_train, X_val, y_train, y_val) = train_test_split(x_train, y_train,	test_size=0.2, random_state=42)

### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [0]:
#Commenting out below piece of code as we are not targetting to submit our Model's predictions on Kaggle competition
#temp_df = pd.read_csv(project_path + 'sample_submission.csv')
#test_img = temp_df['id']
#del temp_df

Run the below code to load the test image files in x_test_feature

In [0]:
#Commenting out below piece of code as we are not targetting to submit our Model's predictions on Kaggle competition
#x_test_feature = []
#i = 0 # initialisation
#for f in tqdm(test_img.values): # f for format ,jpg
#    img = cv2.imread(project_path + 'test/{}.jpg'.format(f), 1)
#    img_resize = cv2.resize(img, (img_rows, img_columns)) 
#    x_test_feature.append(img_resize)

Normalize the test data and convert it into 4 dimensions

In [0]:
#Commenting out below piece of code as we are not targetting to submit our Model's predictions on Kaggle competition
#x_test_feature = np.array(x_test_feature)
#x_test_feature = x_test_feature / 255
#
#x_test_feature = np.reshape(x_test_feature, (len(x_test_feature), img_rows, img_columns, num_channels))

### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [0]:
#Deleting unwanted variables
del labels_df
del labels
del x_train
del i
del breed
del train_img
del train_img_resize

In [0]:
import tensorflow as tf

tf.keras.backend.clear_session()

model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(32,
                                 kernel_size=(5,5),
                                 activation='relu',
                                 input_shape=(img_rows, img_columns, num_channels, )))

model.add(tf.keras.layers.Conv2D(32,
                                 kernel_size=(3,3),
                                 activation='relu'))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(256, activation='relu'))

model.add(tf.keras.layers.Dense(120, activation='softmax'))

In [0]:
y_train = pd.get_dummies(y_train)
y_val = pd.get_dummies(y_val)

In [0]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 124, 124, 32)      2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 122, 122, 32)      9248      
_________________________________________________________________
flatten (Flatten)            (None, 476288)            0         
_________________________________________________________________
dense (Dense)                (None, 256)               121929984 
_________________________________________________________________
dense_1 (Dense)              (None, 120)               30840     
Total params: 121,972,504
Trainable params: 121,972,504
Non-trainable params: 0
_________________________________________________________________


### Use batch_size = 128 and epochs = 10 and execute the model

In [24]:
model.fit(X_train, y_train,          
          validation_data=(X_val, y_val),
          epochs=10,
          batch_size=128)

Epoch 1/10
64/64 [==============================] - 20s 317ms/step - loss: 6.2816 - accuracy: 0.0114 - val_loss: 4.7685 - val_accuracy: 0.0181
Epoch 2/10
64/64 [==============================] - 19s 296ms/step - loss: 4.5338 - accuracy: 0.0585 - val_loss: 4.7119 - val_accuracy: 0.0269
Epoch 3/10
64/64 [==============================] - 19s 296ms/step - loss: 2.8064 - accuracy: 0.3834 - val_loss: 5.6611 - val_accuracy: 0.0225
Epoch 4/10
64/64 [==============================] - 19s 298ms/step - loss: 0.6666 - accuracy: 0.8605 - val_loss: 8.9726 - val_accuracy: 0.0259
Epoch 5/10
64/64 [==============================] - 19s 297ms/step - loss: 0.1302 - accuracy: 0.9817 - val_loss: 9.7725 - val_accuracy: 0.0279
Epoch 6/10
64/64 [==============================] - 19s 298ms/step - loss: 0.0656 - accuracy: 0.9933 - val_loss: 10.7911 - val_accuracy: 0.0352
Epoch 7/10
64/64 [==============================] - 19s 298ms/step - loss: 0.0331 - accuracy: 0.9966 - val_loss: 10.8887 - val_accuracy: 0.03

#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


In [0]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20,
                                                                width_shift_range=0.2,
                                                                height_shift_range=0.2,
                                                                horizontal_flip=True)

In [0]:
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

In [0]:
train_generator = train_datagen.flow(X_train, y_train, batch_size=128)
val_generator = val_datagen.flow(X_val, y_val, batch_size=128)

### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

In [28]:
model.fit_generator(train_generator,
                    epochs = 10,
                    steps_per_epoch = len(X_train) // 128,
                    validation_data = val_generator,
                    validation_steps = len(X_val) // 128)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
63/63 [==============================] - 31s 491ms/step - loss: 4.9606 - accuracy: 0.0140 - val_loss: 4.7556 - val_accuracy: 0.0198
Epoch 2/10
63/63 [==============================] - 31s 489ms/step - loss: 4.7597 - accuracy: 0.0140 - val_loss: 4.7478 - val_accuracy: 0.0177
Epoch 3/10
63/63 [==============================] - 31s 495ms/step - loss: 4.7231 - accuracy: 0.0179 - val_loss: 4.7117 - val_accuracy: 0.0193
Epoch 4/10
63/63 [==============================] - 31s 499ms/step - loss: 4.6884 - accuracy: 0.0211 - val_loss: 4.7291 - val_accuracy: 0.0182
Epoch 5/10
63/63 [==============================] - 31s 490ms/step - loss: 4.6947 - accuracy: 0.0193 - val_loss: 4.6679 - val_accuracy: 0.0193
Epoch 6/10
63/63 [==============================] - 31s 493ms/step - loss: 4.6807 - accuracy: 0.0219 - val_loss: 4.6524 - val_accuracy: 0.0266
Epoch 7/10
63/63 [==============================] - 31s 492ms/step

# Model accuracy is still poor!!!

In [0]:
tf.keras.backend.clear_session()

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [0]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
# Instantiate the model with the pre-trained weights (no top)
base_model= VGG16(weights=(project_path+'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),
                 include_top=False, pooling='avg')

Print the summary of the base_model

In [31]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

In [0]:
x = tf.keras.layers.Flatten()(base_model.output)

x = tf.keras.layers.Dense(1024, activation='relu')(x)

x = tf.keras.layers.Dense(256, activation='relu')(x)

x = tf.keras.layers.Dense(120, activation='softmax')(x)

final_model = tf.keras.models.Model(inputs=base_model.input, #Pre-trained model input as input layer
                                    outputs=x) #Output layer added

### Make all the layers in the base_model (VGG16) to be non-trainable

In [0]:
#Set pre-trained model layers to not trainable
for layer in final_model.layers[:20]:
    layer.trainable = False

In [34]:
final_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

Try to get training and validation accuracy to be more than 90%

In [0]:
final_model.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

In [36]:
final_model.fit_generator(train_generator,
                    epochs = 10,
                    steps_per_epoch = len(X_train) // 128,
                    validation_data = val_generator,
                    validation_steps = len(X_val) // 128)

Epoch 1/10
63/63 [==============================] - 40s 638ms/step - loss: 4.7679 - accuracy: 0.0186 - val_loss: 4.6904 - val_accuracy: 0.0339
Epoch 2/10
63/63 [==============================] - 36s 572ms/step - loss: 4.5450 - accuracy: 0.0468 - val_loss: 4.3211 - val_accuracy: 0.0630
Epoch 3/10
63/63 [==============================] - 37s 580ms/step - loss: 4.1316 - accuracy: 0.0885 - val_loss: 3.9685 - val_accuracy: 0.0964
Epoch 4/10
63/63 [==============================] - 36s 576ms/step - loss: 3.8157 - accuracy: 0.1190 - val_loss: 3.8149 - val_accuracy: 0.1063
Epoch 5/10
63/63 [==============================] - 37s 579ms/step - loss: 3.5993 - accuracy: 0.1587 - val_loss: 3.5355 - val_accuracy: 0.1526
Epoch 6/10
63/63 [==============================] - 36s 569ms/step - loss: 3.4759 - accuracy: 0.1763 - val_loss: 3.4466 - val_accuracy: 0.1734
Epoch 7/10
63/63 [==============================] - 36s 577ms/step - loss: 3.3579 - accuracy: 0.1924 - val_loss: 3.3975 - val_accuracy: 0.1792

In [37]:
final_model.fit_generator(train_generator,
                    epochs = 50,
                    steps_per_epoch = len(X_train) // 128,
                    validation_data = val_generator,
                    validation_steps = len(X_val) // 128,
                    initial_epoch = 10)

Epoch 11/50
63/63 [==============================] - 36s 565ms/step - loss: 3.1236 - accuracy: 0.2385 - val_loss: 3.2912 - val_accuracy: 0.1964
Epoch 12/50
63/63 [==============================] - 36s 574ms/step - loss: 3.0461 - accuracy: 0.2553 - val_loss: 3.2761 - val_accuracy: 0.2115
Epoch 13/50
63/63 [==============================] - 36s 570ms/step - loss: 3.0347 - accuracy: 0.2479 - val_loss: 3.2615 - val_accuracy: 0.2094
Epoch 14/50
63/63 [==============================] - 35s 557ms/step - loss: 2.9394 - accuracy: 0.2780 - val_loss: 3.2857 - val_accuracy: 0.2203
Epoch 15/50
63/63 [==============================] - 36s 573ms/step - loss: 2.9522 - accuracy: 0.2707 - val_loss: 3.2388 - val_accuracy: 0.2255
Epoch 16/50
63/63 [==============================] - 36s 577ms/step - loss: 2.8848 - accuracy: 0.2797 - val_loss: 3.1727 - val_accuracy: 0.2266
Epoch 17/50
63/63 [==============================] - 36s 570ms/step - loss: 2.8655 - accuracy: 0.2921 - val_loss: 3.2084 - val_accuracy:

In [0]:
#Adding BatchNormalization Layer & Dropout Layers before Dense layers to check if the model's performance improves

x = tf.keras.layers.BatchNormalization()(base_model.output)

x = tf.keras.layers.Dropout(0.25)(x)

x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.Dense(1024, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Dropout(0.25)(x)

x = tf.keras.layers.Dense(256, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Dropout(0.25)(x)

x = tf.keras.layers.Dense(120, activation='softmax')(x)

final_model2 = tf.keras.models.Model(inputs=base_model.input, #Pre-trained model input as input layer
                                    outputs=x) #Output layer added

In [0]:
#Set pre-trained model layers to not trainable
for layer in final_model2.layers[:20]:
    layer.trainable = False

In [40]:
final_model2.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0   

In [0]:
final_model2.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

In [42]:
final_model2.fit_generator(train_generator,
                    epochs = 50,
                    steps_per_epoch = len(X_train) // 128,
                    validation_data = val_generator,
                    validation_steps = len(X_val) // 128)

Epoch 1/50
63/63 [==============================] - 37s 581ms/step - loss: 4.9331 - accuracy: 0.0395 - val_loss: 4.5948 - val_accuracy: 0.0495
Epoch 2/50
63/63 [==============================] - 36s 564ms/step - loss: 4.0944 - accuracy: 0.1039 - val_loss: 4.2900 - val_accuracy: 0.1047
Epoch 3/50
63/63 [==============================] - 36s 569ms/step - loss: 3.7157 - accuracy: 0.1579 - val_loss: 3.9781 - val_accuracy: 0.1333
Epoch 4/50
63/63 [==============================] - 36s 571ms/step - loss: 3.5032 - accuracy: 0.1887 - val_loss: 3.6926 - val_accuracy: 0.1776
Epoch 5/50
63/63 [==============================] - 35s 562ms/step - loss: 3.3389 - accuracy: 0.2126 - val_loss: 3.5106 - val_accuracy: 0.1969
Epoch 6/50
63/63 [==============================] - 35s 561ms/step - loss: 3.1923 - accuracy: 0.2383 - val_loss: 3.3451 - val_accuracy: 0.2245
Epoch 7/50
63/63 [==============================] - 35s 563ms/step - loss: 3.0943 - accuracy: 0.2526 - val_loss: 3.2787 - val_accuracy: 0.2271

In [0]:
#Making last layer of VGG16 trainable to see if it improves the Model performance
final_model2.layers[17].trainable = True

In [0]:
final_model2.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

In [48]:
final_model2.fit_generator(train_generator,
                    epochs = 50,
                    steps_per_epoch = len(X_train) // 128,
                    validation_data = val_generator,
                    validation_steps = len(X_val) // 128)

Epoch 1/50
63/63 [==============================] - 37s 586ms/step - loss: 2.7807 - accuracy: 0.3243 - val_loss: 4.9298 - val_accuracy: 0.1594
Epoch 2/50
63/63 [==============================] - 36s 575ms/step - loss: 2.3826 - accuracy: 0.3853 - val_loss: 4.3930 - val_accuracy: 0.2052
Epoch 3/50
63/63 [==============================] - 36s 574ms/step - loss: 2.1313 - accuracy: 0.4275 - val_loss: 3.7117 - val_accuracy: 0.2401
Epoch 4/50
63/63 [==============================] - 36s 577ms/step - loss: 1.9582 - accuracy: 0.4679 - val_loss: 3.6661 - val_accuracy: 0.2234
Epoch 5/50
63/63 [==============================] - 36s 571ms/step - loss: 1.8868 - accuracy: 0.4878 - val_loss: 3.4706 - val_accuracy: 0.2448
Epoch 6/50
63/63 [==============================] - 36s 566ms/step - loss: 1.7659 - accuracy: 0.5147 - val_loss: 3.6438 - val_accuracy: 0.2359
Epoch 7/50
63/63 [==============================] - 36s 572ms/step - loss: 1.7042 - accuracy: 0.5201 - val_loss: 3.3691 - val_accuracy: 0.2542

After allowing to train last layer of VGG16, we can see that the Training accuracy increased to around 80% but validation accuracy is still hovering around 29 to 30%.

That means the model is overfitting on Training data but not performing well in general.

To address this problem, we had already included BatchNormalization & Dropout Layers, but probably we need to increase the Dropout rate from 0.25 to 0.5 may be & check.

Also, we can try making few more layers trainable because VGG16 was trained on different Objects out of which one might be Dog but definitely not it's breeds. But, that would increase the training time significantly & it is not easy to manage it on Google Colab because it gets disconnected from the GPU after inactivity & hence it cannot be simply allowed to leave it for training for huge amount of time.

Another approach could be try using some advanced models like Googlenet, Resnet or Mobilenet.